In [ ]:
!git clone https://github.com/NearANDfar13/NLP_Winter_2024.git

Cloning into 'NLP_Winter_2024'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 87 (delta 38), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (87/87), 2.63 MiB | 2.07 MiB/s, done.
Resolving deltas: 100% (38/38), done.


In [ ]:
#Import the dataset "San Fran Small Sample"
import pandas as pd

df = pd.read_excel('/content/NLP_Winter_2024/NA_San_Fran_Small_Sample.xlsx')

#DataFrame 'df'
df.head()

,ID,Day,Short_Name,Full_Name,Place,Frequency,Publication,Text,Page
0,2975858-0,19/04/1906,Aachener Anzeiger,Aachener Anzeiger : politisches Tageblatt : be...,Aachen,täglich,1885-1943,0=Erdbeben und Stadtbrand in SauFrancisco.New=...,?
1,2797055-3,19/04/1906,Badische Presse,Badische Presse : Generalanzeiger der Residenz...,Karlsruhe,täglich,1890-1944,Die Marinestationen am Süllen Ozean versuchten...,4
2,2436020-X,19/04/1906,Berliner Börsen-Zeitung,"Berliner Börsen-Zeitung, Tageszeitung für Poli...",Berlin,täglich,NaN,richten lassen die Verwüstung noch größer ersc...,5
3,3144747-8,19/04/1906,Cannstatter Zeitung,"Cannstatter Zeitung, allgemeiner Anzeiger für ...","Stuttgart, Cannstatt, Stuttgart-Bad Cannstatt,...",täglich,NaN,"Cincinnati, 18. April. Einem hiesigen Blatte w...",1
4,3144747-8,19/04/1906,Cannstatter Zeitung,"Cannstatter Zeitung, allgemeiner Anzeiger für ...","Stuttgart, Cannstatt, Stuttgart-Bad Cannstatt,...",täglich,NaN,"Newyork, 19. April. Ten letzten Nachrichten an...",3


In [5]:
!pip install sentence-transformers

In [6]:

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
from sentence_transformers import SentenceTransformer
from collections import Counter

def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'[^a-zäöüß\s]', '', text)
    return text

def get_unique_words(text):
    words = text.split()
    return list(set(words))

def find_similar_words(df, target_word, model, top_n=40):
    # Preprocess the text
    df['processed_text'] = df['Text'].apply(preprocess_text)

    # Get unique words from all texts
    all_words = []
    for text in df['processed_text']:
        all_words.extend(get_unique_words(text))

    # Get unique words and their frequencies
    word_freq = Counter(all_words)
    unique_words = list(word_freq.keys())

    print(f"Number of unique words: {len(unique_words)}")

    # Encode the target word and unique words
    target_embedding = model.encode([target_word])
    word_embeddings = model.encode(unique_words)

    # Calculate similarities
    similarities = cosine_similarity(target_embedding, word_embeddings)[0]

    # Create a DataFrame with words and their similarities
    word_sim_df = pd.DataFrame({
        'word': unique_words,
        'similarity': similarities
    })

    # Sort by similarity and get top N results
    top_similar = word_sim_df.sort_values('similarity', ascending=False).head(top_n)

    return top_similar

# Load the pre-trained multilingual model
print("Loading the sentence transformer model...")
model = SentenceTransformer('sentence-transformers/LaBSE')
print("Model loaded successfully.")

target_word = "Feuer"

print(f"\nFinding words similar to '{target_word}'...")
similar_words = find_similar_words(df, target_word, model)

print("\nMost similar words:")
print(similar_words)

Loading the sentence transformer model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

...

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense%2Fconfig.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

Model loaded successfully.

Finding words similar to 'Feuer'...
Number of unique words: 4775

Most similar words:
                    word  similarity
49                 feuer    0.844023
318              flammen    0.800606
495               brennt    0.767372
2205             brennen    0.765218
2841          brennenden    0.713895
3802          brennendem    0.710226
1001           feuerwehr    0.710176
1111              feuers    0.709853
2535        feuersdrunßt    0.708355
2824       feuersbrünste    0.698701
1019           brennende    0.697815
1530         flammenmeer    0.687916
4237      feuersbrünsten    0.678809
38              feuerhat    0.677500
1799         feuersbrunt    0.660967
1096           verbrannt    0.656472
2073           entzündet    0.655061
4683          brennpunkt    0.647826
637         feuersbrunst    0.647786
198       flammengetötet    0.643386
1599        feuerschaden    0.641768
682          verbrannten    0.629941
3786        feuersbrnnst    0.62559